## Shelf Space Optimization

Problem: You are provided a dataset which provides sales data for three different products sold in an apparel retailer. These products are three different types of jeans sold at this retailer - Colored denim, Wide Leg, High-Rise.You are required to determine an ideal shelf allocation across these three products. The total shelf spaces available are 60. You are required to provide at least 5 shelf spaces to each product. Based on today’s class, please frame a problem, and if possible attempt a solution to this problem in a tool of your choice (& comfort).

In [61]:
#Objective: 
#    Maximize: Sales = sum(Sales1+Sales2+Sales3) 
#    Sales(n) = baseline + is_discount X a1 + is_weekend * a2+space * a3 + price * a4 (n =1,2,3)
#Constraint: 
#    x1+x2+x3<=60
#    x1<=5
#    x2<=5
#    x3<=5

### import the packages

In [62]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import cvxpy as cvx

### fit elasticity model to get the parameters for 3 products using OLS

In [82]:
def get_params(sheet):
    df= pd.read_excel("data.xlsx",sheet_name=sheet )
    df["day"] = df["Day of the Week"].apply(lambda x: 1 if x in ["Sa","Su"] else 0)
    def dis(x):
        if x["Markdown"]+x["Clearance"]+x["Discount"] >=1:
            return 1
        else: return 0
    
    df["dis"]= df.apply(dis,axis=1)
    df["space"] = (df["Shelf Spaces"])
    df["sales"] = (df["Sales"])
    df["price"] = (df["Average Price"])
    fit = sm.ols(formula="sales ~ space + price + day + dis", data=df).fit()
    result = fit.params
    return result

In [83]:
m1= get_params("colored")
m1

Intercept    74858.667465
space          794.155733
price        -1681.235429
day           3608.791608
dis         -14748.524292
dtype: float64

In [84]:
m2 =get_params("high-rise")
m2

Intercept    25234.825774
space         2019.413986
price          102.384814
day          23904.151448
dis           -988.901138
dtype: float64

In [85]:
m3= get_params("wide-leg")
m3

Intercept    3998.829234
space        1783.298828
price        -113.459609
day         -2102.421183
dis         -3898.812391
dtype: float64

### optimize with integer programming

In [86]:
def optimize(is_weekend, is_discount_1, is_discount_2, is_discount_3, price1, price2, price3, m1, m2, m3):
    x1 = cvx.Int()
    x2 = cvx.Int()
    x3 = cvx.Int()
    obj = cvx.Maximize(m1[0]+m1[1]*x1+m1[2]*price1+m1[3]*is_weekend+m1[4]*is_discount_1
    +m2[0]+m2[1]*x2+m2[2]*price2+m2[3]*is_weekend+m2[4]*is_discount_2
    +m3[0]+m3[1]*x3)
    con = [x1+x2+x3<=60, x1>=5, x2>=5, x3>=5]
    prob = cvx.Problem(obj, con)
    result = prob.solve()
    return x1.value,x2.value,x3.value,obj.value

### test

In [87]:
optimize(1, 0, 1, 1, 30, 20, 40, m1, m2, m3)

(5.0000000023752103,
 49.999999991771986,
 5.0000000043909862,
 181336.44561368937)